# Transformation of ****.nnet*** (Stanford) file into ****.pb*** file (TensorFlow)

## 1 - Librairies

In [1]:
# import tensorflow as tf
import numpy as np 
import sys
from tensorflow.python.framework import graph_util
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## 2 - Define readNNet, normalizeNNet, writeNNET functions

In [3]:
def readNNet(nnetFile, withNorm=False):
    '''
    Read a .nnet file and return list of weight matrices and bias vectors
    
    Inputs:
        nnetFile: (string) .nnet file to read
        withNorm: (bool) If true, return normalization parameters
    
    Returns: 
        weights: List of weight matrices for fully connected network
        biases: List of bias vectors for fully connected network
    '''        
    # Open NNet file
    f = open(nnetFile,'r')
    
    # Skip header lines
    line = f.readline()
    while line[:2]=="//":
        line = f.readline()
        
    # Extract information about network architecture
    record = line.split(',')
    numLayers   = int(record[0])
    inputSize   = int(record[1])

    line = f.readline()
    record = line.split(',')
    layerSizes = np.zeros(numLayers+1,'int')
    for i in range(numLayers+1):
        layerSizes[i]=int(record[i])

    # Skip extra obsolete parameter line
    f.readline()
    
    # Read the normalization information
    line = f.readline()
    inputMins = [float(x) for x in line.strip().split(",") if x]

    line = f.readline()
    inputMaxes = [float(x) for x in line.strip().split(",") if x]

    line = f.readline()
    means = [float(x) for x in line.strip().split(",") if x]

    line = f.readline()
    ranges = [float(x) for x in line.strip().split(",") if x]

    # Read weights and biases
    weights=[]
    biases = []
    for layernum in range(numLayers):

        previousLayerSize = layerSizes[layernum]
        currentLayerSize = layerSizes[layernum+1]
        weights.append([])
        biases.append([])
        weights[layernum] = np.zeros((currentLayerSize,previousLayerSize))
        for i in range(currentLayerSize):
            line=f.readline()
            aux = [float(x) for x in line.strip().split(",")[:-1]]
            for j in range(previousLayerSize):
                weights[layernum][i,j] = aux[j]
        #biases
        biases[layernum] = np.zeros(currentLayerSize)
        for i in range(currentLayerSize):
            line=f.readline()
            x = float(line.strip().split(",")[0])
            biases[layernum][i] = x

    f.close()
    
    if withNorm:
        return weights, biases, inputMins, inputMaxes, means, ranges
    return weights, biases

In [4]:
def normalizeNNet(readNNetFile, writeNNetFile=None):
    weights, biases, inputMins, inputMaxes, means, ranges = readNNet(readNNetFile,withNorm=True)
    
    numInputs = weights[0].shape[1]
    numOutputs = weights[-1].shape[0]
    
    # Adjust weights and biases of first layer
    for i in range(numInputs): weights[0][:,i]/=ranges[i]
    biases[0]-= np.matmul(weights[0],means[:-1])
    
    # Adjust weights and biases of last layer
    weights[-1]*=ranges[-1]
    biases[-1] *= ranges[-1]
    biases[-1] += means[-1]
    
    # Nominal mean and range vectors
    means = np.zeros(numInputs+1)
    ranges = np.ones(numInputs+1)
    
    if writeNNetFile is not None:
        writeNNet(weights,biases,inputMins,inputMaxes,means,ranges,writeNNetFile)
        return None
    return weights, biases

In [5]:
def writeNNet(weights,biases,inputMins,inputMaxes,means,ranges,fileName):
    '''
    Write network data to the .nnet file format
    Args:
        weights (list): Weight matrices in the network order 
        biases (list): Bias vectors in the network order
        inputMins (list): Minimum values for each input
        inputMaxes (list): Maximum values for each input
        means (list): Mean values for each input and a mean value for all outputs. Used to normalize inputs/outputs
        ranges (list): Range values for each input and a range value for all outputs. Used to normalize inputs/outputs
        fileName (str): File where the network will be written
    '''
    #Open the file we wish to write
    with open(fileName,'w') as f2:

        #####################
        # First, we write the header lines:
        # The first line written is just a line of text
        # The second line gives the four values:
        #     Number of fully connected layers in the network
        #     Number of inputs to the network
        #     Number of outputs from the network
        #     Maximum size of any hidden layer
        # The third line gives the sizes of each layer, including the input and output layers
        # The fourth line gives an outdated flag, so this can be ignored
        # The fifth line specifies the minimum values each input can take
        # The sixth line specifies the maximum values each input can take
        #     Inputs passed to the network are truncated to be between this range
        # The seventh line gives the mean value of each input and of all outputs
        # The eighth line gives the range of each input and of all outputs
        #     These two lines are used to map raw inputs to the 0 mean, unit range of the inputs and outputs
        #     used during training
        # The ninth line begins the network weights and biases
        ####################
        f2.write("// Neural Network File Format by Kyle Julian, Stanford 2016\n")

        #Extract the necessary information and write the header information
        numLayers = len(weights)
        inputSize = weights[0].shape[1]
        outputSize = len(biases[-1])
        maxLayerSize = inputSize
        
        # Find maximum size of any hidden layer
        for b in biases:
            if len(b)>maxLayerSize :
                maxLayerSize = len(b)

        # Write data to header 
        f2.write("%d,%d,%d,%d,\n" % (numLayers,inputSize,outputSize,maxLayerSize) )
        f2.write("%d," % inputSize )
        for b in biases:
            f2.write("%d," % len(b) )
        f2.write("\n")
        f2.write("0,\n") #Unused Flag

        # Write Min, Max, Mean, and Range of each of the inputs and outputs for normalization
        f2.write(','.join(str(inputMins[i])  for i in range(inputSize)) + ',\n') #Minimum Input Values
        f2.write(','.join(str(inputMaxes[i]) for i in range(inputSize)) + ',\n') #Maximum Input Values                
        f2.write(','.join(str(means[i])      for i in range(inputSize+1)) + ',\n') #Means for normalizations
        f2.write(','.join(str(ranges[i])     for i in range(inputSize+1)) + ',\n') #Ranges for noramlizations

        ##################
        # Write weights and biases of neural network
        # First, the weights from the input layer to the first hidden layer are written
        # Then, the biases of the first hidden layer are written
        # The pattern is repeated by next writing the weights from the first hidden layer to the second hidden layer,
        # followed by the biases of the second hidden layer.
        ##################
        for w,b in zip(weights,biases):
            for i in range(w.shape[0]):
                for j in range(w.shape[1]):
                    f2.write("%.5e," % w[i][j]) #Five digits written. More can be used, but that requires more more space.
                f2.write("\n")
                
            for i in range(len(b)):
                f2.write("%.5e,\n" % b[i]) #Five digits written. More can be used, but that requires more more space.

## 3 - Define transformation functions

In [6]:
def nnet2pb(nnetFile, pbFile="", output_node_names = "y_out", normalizeNetwork=False):
    '''
    Read a .nnet file and create a frozen Tensorflow graph and save to a .pb file
    
    Args:
        nnetFile (str): A .nnet file to convert to Tensorflow format
        pbFile (str, optional): Name for the created .pb file. Default: ""
        output_node_names (str, optional): Name of the final operation in the Tensorflow graph. Default: "y_out"
    '''
    if normalizeNetwork:
        weights, biases = normalizeNNet(nnetFile)
    else:
        weights, biases = readNNet(nnetFile)
    inputSize = weights[0].shape[1]
    
    # Default pb filename if none are specified
    if pbFile=="":
        pbFile = nnetFile[:-4]+'pb'
    
    # Reset tensorflow and load a session using only CPUs
    # tf.compat.v1.reset_default_graph() ### COMPAT.V1.
    # sess = tf.compat.v1.Session() ### COMPAT.V1.
    tf.reset_default_graph() ### COMPAT.V1.
    sess = tf.Session() ### COMPAT.V1.

    # Define model and assign values to tensors
    currentTensor = tf.placeholder(tf.float32, [None, inputSize],name='input')
    for i in range(len(weights)):
        W = tf.get_variable("W%d"%i, shape=weights[i].T.shape)
        b = tf.get_variable("b%d"%i, shape=biases[i].shape)
        
        # Use ReLU for all but last operation, and name last operation to desired name
        if i!=len(weights)-1:
            currentTensor = tf.nn.relu(tf.matmul(currentTensor ,W) + b)
        else:
            currentTensor =  tf.add(tf.matmul(currentTensor ,W), b,name=output_node_names)

        # Assign values to tensors
        sess.run(tf.assign(W,weights[i].T))
        sess.run(tf.assign(b,biases[i]))
    
    # Freeze the graph to write the pb file
    freeze_graph(sess,pbFile,output_node_names)

In [7]:
def freeze_graph(sess, output_graph_name, output_node_names):
    '''
    Given a session with a graph loaded, save only the variables needed for evaluation to a .pb file
    
    Args:
        sess (tf.session): Tensorflow session where graph is defined
        output_graph_name (str): Name of file for writing frozen graph
        output_node_names (str): Name of the output operation in the graph, comma separated if there are multiple output operations
    '''
    
    input_graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(
        sess,                        # The session is used to retrieve the weights
        input_graph_def,             # The graph_def is used to retrieve the nodes 
        output_node_names.split(",") # The output node names are used to select the useful nodes
    ) 

    # Finally we serialize and dump the output graph to the file
    with tf.gfile.GFile(output_graph_name, "w") as f:
        f.write(output_graph_def.SerializeToString())

## 4 - Transform to *.pb

In [8]:
# Read user inputs and run writePB function

nnetFile = "ACAS_Xu_NNet/" + "ACASXU_1_1.nnet"

nnet2pb(nnetFile,"","y_out")
print("> TF network saved as *.pb file !")

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
> TF network saved as *.pb file !


## 5 - Transform in *.h5

In [9]:
tfFile = nnetFile[:-4] + "pb"
print(tfFile)

ACAS_Xu_NNet/ACASXU_1_1.pb


In [10]:
from tensorflow.keras.models import load_model

In [13]:
model = tf.keras.models.load_model("saved_model.pb")

OSError: SavedModel file does not exist at: saved_model.pb/{saved_model.pbtxt|saved_model.pb}